# This notebook creates the flight plans for a set of OD pairs stored in a csv file
The .csv file must contain the following fields for each drone:
 - Origin
 - Destination
 - Departure time
 - Type of drone
 
The notebook will provide scenario files for each drone.

TODO: include strategic deconfliction

In [13]:
"""
We define a default path where the modules developed in USEPE are stored
"""

default_path = r"C:\workspace3\bluesky"

"""
We define a second default path where the scenarios are stored
"""

default_path2 = r"C:\workspace3\scenarios-exercise1\scenario\USEPE\exercise_1"


In [14]:
import os
import sys
import configparser
import pickle
import osmnx as ox
import random
import pandas as pd
import datetime

os.chdir(default_path)
from usepe.city_model.path_planning import trajectoryCalculation
from usepe.city_model.scenario_definition import createFlightPlan
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.utils import read_my_graphml, layersDict
from usepe.city_model.city_graph import cityGraph

ModuleNotFoundError: No module named 'usepe'

Configuration file¶

We create a config file with the following fields:

[City]

mode = rectangle

#hannover_lat = 52.376

#hannover_lon = 9.76

#zone_size = 1000

hannover_lat_min = 52.297839999994714

hannover_lat_max = 52.438260000005286

hannover_lon_min = 9.643080000001035

hannover_lon_max = 9.844349999998968

import = True

imported_graph_path = ./data/exercise_1_reference.graphml

[Layers]
number_of_layers = 9

layer_width = 25

[BuildingData]

lat_min = 52.297839999994714

lat_max = 52.438260000005286

lon_min = 9.643080000001035

lon_max = 9.844349999998968

divisions = 20

directory_hannover = C:\workspace3\bluesky_usepe_git\usepe\city_model\data\CityGML_LoD1

[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2

[Outputs]

graph_path = ./data/exercise_1_reference.graphml

[Segments]

import = True

path = ./data/reference_segments.pkl

[Corridors]

corridors =

altitude = 250

delta_z = 25

speed = 50

acceleration_length = 50

file_path_corridors = ./data/usepe-hannover-corridors.geojson

[Strategic_Deconfliction]

ratio = 3

delay = 60

In [ ]:
# -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = default_path2 + r"\settings_exercise_1_reference.cfg"
config = configparser.ConfigParser()
config.read( config_path )

# Import the graph

In [ ]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined with the city section of the
configuration file.
"""
# City
os.chdir(default_path2)
if config['City'].getboolean( 'import' ):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml( filepath )
    G = MultiDiGrpah3D( G )
    # fig, ax = ox.plot_graph( G )
else:
    print('Check the path to the graph file')
    G = cityGraph( config )

# Flight plan creation function

In [ ]:
def createFlightPlan(path_csv, G, config):
    '''
    This function reads the data stored in a csv file and creates the flight plans accordingly
    '''
    
    #Open csv
    data_frame = pd.read_csv(path_csv)
    
    # Create the path where the scenarios are stored
    scenario_general_path_base = path_csv[:-4]
    
    # General scenario that calls all drone scenarios
    scenario_general_path = scenario_general_path_base + '\\' + 'scenario_background_traffic.scn'
    if not os.path.exists( os.path.dirname( scenario_general_path ) ):
        os.makedirs( os.path.dirname( scenario_general_path ) )
    scenario_general_file = open( scenario_general_path, 'w' )
    
    # basename for the drones
    base_name = 'U'
    
    for n in range(data_frame.shape[0]):
        
        orig = data_frame['origin'][n]
        dest = data_frame['destination'][n]
    
        name = base_name + str( n+1 )  # drone name
        travel_time, route = trajectoryCalculation( G, orig, dest )

        #print( 'The travel time of the route is {0}'.format( travel_time ) )
        #print( 'The route is {0}'.format( route ) )

        # Path Planning
        ac = {'id': name, 
              'type': data_frame['drone'][n], 
              'accel': 3.5,  # TODO: change with the values of the specific model
              'v_max': 18,  # TODO: change with the values of the specific model
              'vs_max': 5 }  # TODO: change with the values of the specific model
        departure_time = data_frame['departure'][n]

        scenario_path = scenario_general_path_base + '\\' + 'scenario_background_traffic_drone' + str( n ) + '.scn'

        scenario_file = open( scenario_path, 'w' )
        
        layers_dict = layersDict( config )
        createFlightPlan( route, ac, departure_time, G, layers_dict, scenario_file )
        scenario_file.close()

        scenario_general_file.write( '00:00:00.00 > PCALL ' + scenario_path + ' REL' + '\n' )
    scenario_general_file.close()
    print('Output scenario stored in the following directory {0}'.format(scenario_general_path_base))

# Determine the path to the csv and create the flight plans

In [ ]:
path_csv = "C:/workspace3/scenarios-exercise1/scenario/USEPE/exercise_1/data/background_traffic_0-0964_2384_7200.csv"

In [ ]:
createFlightPlan(path_csv, G, config)